In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install inflect

In [2]:
import multiprocessing as mp
mp.set_start_method("spawn", force=True)

import sys
import os

sys.path.append(os.path.abspath(os.path.dirname('/home/jdd5089/projects/llm-fsp/finetune-notebooks/json-existing/phi-4/vllm-inference.ipynb')))


In [6]:
# vLLM offline inference with LoRA adapter on phi-4

from vllm import LLM, SamplingParams
import pandas as pd

MAX_TOKENS = 4096  # Maximum tokens for generation
LORA_PATH = "phi-4-fsp-ft-cand"  # Path to LoRA adapter

frame_descriptions = pd.read_csv('/home/jdd5089/projects/llm-fsp/finetune-notebooks/json-existing/frame_descriptions_json.csv')
frame_descriptions_mapping = frame_descriptions[['name', 'description']].set_index('name').to_dict()['description']


system_prompt = """### Task:
You are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. 

### Notes:
- Return the tagged sentence in a ```json ``` code block.
- Texts must not overlap.
"""

user_prompt = """### Frame Information:
{frame_info}

### Input:
{input_sentence}
"""

# Path to base model and LoRA adapter
base_model_path = "unsloth/phi-4-unsloth-bnb-4bit"
lora_adapter_path = "./phi-4-fsp-ft-cand"


# Initialize vLLM with LoRA adapter
llm = LLM(
    model=base_model_path,
    enable_lora=True,
    dtype="auto",  # or "float16" for GPU, "bfloat16" if supported
    device="cuda"  # or "cpu" if no GPU
)

INFO 08-07 13:09:42 [config.py:717] This model supports multiple tasks: {'classify', 'generate', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 08-07 13:09:43 [config.py:830] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-07 13:09:43 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 08-07 13:09:43 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='unsloth/phi-4-unsloth-bnb-4bit', speculative_config=None, tokenizer='unsloth/phi-4-unsloth-bnb-4bit', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto,  device_config=

Process EngineCore_0:
Traceback (most recent call last):
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 400, in run_engine_core
    raise e
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 329, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/home/jdd5089/miniconda3/envs/llm/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 64, in __init__
    self.mo

RuntimeError: Engine core initialization failed. See root cause above.

In [ ]:
from candidate_identifier import LexicalUnitManager

lum = LexicalUnitManager()
lum.load_lus()

def get_candidates(text: str) -> list:
    """
    Get candidate lexical units from the text.

    Args:
        text (str): The input text to analyze.

    Returns:
        list: A list of dictionaries, each containing a target-highlighted string and 
        its corresponding candidate frames.
    """
    candidates = lum.lookup_lus(text)
    
    result = []
    for span, cand_frames in candidates.items():
        highlighted_text = f"{text[:span[0]]}**{text[span[0]:span[1]]}**{text[span[1]:]}"
        result.append({
            "target": highlighted_text,
            "candidates": cand_frames
        })
    
    return result

# Example usage
# text = "The bank will not lend money to the company."
# candidates = get_candidates(text)


In [ ]:
from vllm.lora.request import LoRARequest
fsp_lora = LoRARequest("fsp_lora", 1, LORA_PATH)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-4")

def frame_semantic_parse(text: str, tie_breaker = "random") -> dict:
    """
    Perform frame semantic parsing on the input text using vLLM with LoRA adapter.

    Args:
        text (str): The input text to analyze.

    """

    candidates = get_candidates(text)
    
    # Create prompts for each candidate
    user_prompts = []
    for candidate in candidates:
        target = candidate['target']
        for frame in candidate['candidates']:
            frame_info = frame_descriptions_mapping[frame]
            prompt = user_prompt.format(frame_info=frame_info, input_sentence=target)
            user_prompts.append(prompt)
    
    messages = [[{"role": "system", "content": system_prompt}, 
                {"role": "user", "content": user_prompt}] for user_prompt in user_prompts]
    
    messages = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    print(messages)
    
    sampling_params = SamplingParams(
        max_tokens=MAX_TOKENS,
    )

    # Run inference
    responses = llm.generate(
        messages,
        sampling_params=sampling_params,
        lora_request=fsp_lora
    )

    return responses

# Example usage
while True:
    text = input("Enter a sentence for frame semantic parsing (or 'exit' to quit): ")
    if text.lower() == 'exit':
        break
    if not text.strip():
        print("Empty input. Please enter a valid sentence.")
        continue
    response = frame_semantic_parse(text)

    for i, res in enumerate(response):
        print(f"Response {i+1}:")
        for message in res.outputs:
            if message.text:
                print(message.text)
            else:
                print("No text output.")
        print("\n" + "="*50 + "\n")
    print("Done processing the input.")




In [4]:
messages = ['<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Relational_natural_features\nFrame Definition: The Focal_feature is defined in relation to a Landmark_feature, either as a particular part, or as an immediately bordering entity.\n\nFrame Elements:\nConstituent_parts (Peripheral): Salient parts of the Focal_feature.\nContainer_possessor (Extra-Thematic): The location that the Focal_feature is a part of.\nFormational_cause (Extra-Thematic): Indicates the action (or causer) which brings the features of the Focal_feature about.\nFocal_feature (Core): A stable bounded area defined by its relation to another landform. This FE is incorporated in each LU in this frame.\nName (Extra-Thematic): This FE is used for the Names of Focal_features.\nRelative_location (Peripheral): A place that a Focal_feature is located with respect to.\nType (Peripheral): The Type is the subtype of natural feature.\nDescriptor (Extra-Thematic): Any description, characteristic, or property of the Focal_feature which is not covered by more specific FEs.\nLandmark_feature (Core): The landform with respect to which the Focal_feature is defined.\n\n\n### Input:\nThe **bank** will not lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Businesses\nFrame Definition: A Proprietor owns or runs a Business which provides a Product (which may be goods or services).\n\nFrame Elements:\nProduct (Peripheral): Product identifies the product or service provided by the business.\nProprietor (Peripheral): Proprietor is the owner or chief participant in the business.\nDescriptor (Peripheral): Descriptor indicates other characteristics of the business besides its Product and Proprietor.\nPlace (Peripheral): This FE identifies the Place in which the business is located.\nBusiness (Core): Business identifies the company or establishment. This may be the target itself or the Business_Name if it is an appositive not set off with commas (i.e. restrictive):\nBusiness_name (Peripheral): The name of the Business when it appears in an appositive set off by commas (non-restrictive):\nService_provider (Peripheral): When a Business is described in terms of the professionals who work to produce the product or service, use Service_provider.\n\n\n### Input:\nThe **bank** will not lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', "<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Giving\nFrame Definition: A Donor transfers a Theme from a Donor to a Recipient. This frame includes only actions that are initiated by the Donor (the one that starts out owning the Theme). Sentences (even metaphorical ones) must meet the following entailments: the Donor first has possession of the Theme. Following the transfer the Donor no longer has the Theme and the Recipient does.\n\nFrame Elements:\nDonor (Core): The person that begins in possession of the Theme and causes it to be in the possession of the Recipient .\nRecipient (Core): The entity that ends up in possession of the Theme.\nTheme (Core): The object that changes ownership.\nPlace (Peripheral): The Place where the Donor gives the Theme to the Recipient.\nExplanation (Extra-Thematic): The Explanation for which the Donor gives the Theme to the Recipient.\nTime (Peripheral): The Time is when the Donor gives the Theme to the Recipient.\nPurpose (Peripheral): The Purpose for which the Donor gives the Theme to the Recipient.\nMeans (Peripheral): The Means by which the Donor gives the Theme to the Recipient.\nManner (Peripheral): The Manner is the manner in which the Donor gives the Theme to the Recipient.\nCircumstances (Extra-Thematic): The Circumstances are the conditions under which the Theme is given.\nImposed_purpose (Extra-Thematic): The Recipient's intended purpose for the Theme.\nDepictive (Extra-Thematic): A description of the Donor, Recipient, or Theme given independently of the giving event per se.\nPeriod_of_iterations (Extra-Thematic): The length of time from when the event denoted by the target began to be repeated to when it stopped.\n\n\n### Input:\nThe bank **will** not lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>", '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Documents\nFrame Definition: Words in the frame refer to any Document that has a legal status or conventional social significance. Some Documents empower the Bearer of the Document to execute the Right. Others indicate the Obligation of the Bearer. Still others show the identity or Status of the Bearer.\n\nFrame Elements:\nBearer (Core): The holder of the document.\nIssuer (Core): The authority that issues the document.\nObligation (Core): Obligation is the obligation that is imposed on the Bearer of the Document.\nRight (Core): Right is the right of the Bearer of the Document that is authorized by the Document.\nStatus (Core): Status is the state of the Bearer of the Document indicated by the Document.\nSpecification (Extra-Thematic): A Specification is a condition that applies to the Rights and/or Obligations described in the Document.\nDocument (Core): Document is the piece of paper that has the legal status.\nMedium (Peripheral): The language the Document is written in, or a specification of the kind of object that instantiates it.\nDescriptor (Extra-Thematic): Descriptor covers any characterization or description of the target that is not covered by another more specific FE.\n\n\n### Input:\nThe bank **will** not lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', "<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Desiring\nFrame Definition: An Experiencer desires that an Event occur. (Note that commonly a resultant state of the Event will stand in for the Event.) In some cases, the Experiencer is an active participant in the Event, and in such cases the Event itself is often not mentioned, but rather some Focal_participant which is subordinately involved in the Event. Generally, the use of a word in this frame implies that the specific Event has not yet happened, but that the Experiencer believes that they would be happier if it did. Sometimes the Time_of_event, Purpose_of_event, or the Location_of_event are mentioned without the explicit mention of the Event.\n\nFrame Elements:\nExperiencer (Core): The Experiencer is the person (or sentient being) who wishes for the Event to occur.\nEvent (Core): The change that the Experiencer would like to see.\nFocal_participant (Core): This is the entity that the Experiencer wishes to be affected by some Event.\nDegree (Peripheral): The Degree is a the extent of the Experiencer's Desiring.\nManner (Peripheral): The way in which the Experiencer desires something.\nExplanation (Extra-Thematic): An aspect of the Experiencer that causes a desire.\nPurpose_of_event (Peripheral): The Purpose_of_event is the purpose pertaining to the desired Event.\nTime_of_event (Peripheral): The time that the Event would ideally occur.\nLocation_of_event (Core): The Location_of_event is the place involved in the desired Event.\nTime (Peripheral): The Time is when the Experiencer desires something.\nPlace (Peripheral): Where the event takes place. Note that this FE is of extremely rare occurrence in this frame.\nDuration (Peripheral): The amount of time for which the Experiencer has desired something.\nRole_of_focal_participant (Peripheral): The Role is a role filled by a Focal_participant in the Event that the Experiencer desires.\n\n\n### Input:\nThe bank **will** not lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>", '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Negation\nFrame Definition: This frame models the semantic contribution of negation, in which the profiled content of the Negated_proposition is asserted to contrast incompatibly with a corresponding Factual_situation, which is normally implicit. The Negated_proposition and the Factual_situation share non-focal content, on the basis of which they correspond, and contrast in a subset of their content, the focal content, established with reference to a set of focus constructions not described here. (The focus of negation is often conveyed intonationally in spoken English; in text, however, negative focus must be derived inferentially from contextual reference. A subset of focus constructions will be annotated in the constructicon.)\n\nFrame Elements:\nNegated_proposition (Core): The situation which is presented as false overall. The particular focus of negation which contrasts with the Factual_situation is not separately marked.\nFactual_situation (Core): The situation or part of situation that actually pertains, in contradistinction to the profiled situation in the Negated_proposition that is asserted not to pertain. In spoken language and casual written language, but not in normatively edited written texts, the Factual_situation may be expressed as an unmarked clause immediately preceding or following the Negated_proposition.\n\n\n### Input:\nThe bank will **not** lend money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Lending\nFrame Definition: The Lender gives the Theme to the Borrower with the expectation that the Borrower will return the Theme to the Lender after a Duration of time. This frame differs from the Borrowing frame in that this frame profiles the Lender in active sentences, whereas the Borrowing frame profiles the Borrower.\n\nFrame Elements:\nBorrower (Core): The person or institution who receives the Theme from the Lender for a Duration.\nDuration (Peripheral): The amout of time in which the Borrower has possession of the Theme.\nLender (Core): The person or institution who gives the Theme to the Borrower for a Duration.\nTheme (Core): The object that is transferred from the Lender to the Borrower for a Duration.\nPurpose (Peripheral): The aim of the Lender which they believe will be accomplished by lending the Theme to the Borrower.\nTime (Peripheral): The time when the lending event occurs.\nManner (Peripheral): The way in which the Lender lends the Theme.\nPlace (Peripheral): The location in which the Lender lends the Theme to the Borrower.\n\n\n### Input:\nThe bank will not **lend** money to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Money\nFrame Definition: Money is a medium used for exchanging goods and services. It is in most cases issued by a government (its Creator) in the form of coins and banknotes. Particular amounts of it may have a designated or planned Use, or may have come from some Origin.\n\nFrame Elements:\nMoney (Core): Money is used as a medium of exchange.\nInherent_purpose (Extra-Thematic): The use for which the Money is intended or designed.\nCreator (Extra-Thematic): The government (or possibly other organization) that mints, issues and guarantees the legal use of the Money.\nTime_of_creation (Extra-Thematic): The time at which an Money comes into existence or begins to be circulated.\nName (Extra-Thematic): The term used to refer to the Money.\nType (Extra-Thematic): An indication of the subtype of Money.\nMaterial (Extra-Thematic): Any indication of what makes up the Money, including components, ingredients, etc.\nOrigin (Extra-Thematic): The process or means by which the Money comes to be possessed.\nPossessor (Extra-Thematic): The Possessor has control over the use the Money.\n\n\n### Input:\nThe bank will not lend **money** to the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', "<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Locative_relation\nFrame Definition: A Figure is located relative to a Ground location.\n\nFrame Elements:\nGround (Core): The Ground serves as a basis for describing the location of the Figure.\nFigure (Core): The Figure is perceived as located relative to a certain Ground location. The figure can be an entity or an event.\nDistance (Extra-Thematic): This FE identifies the Distance between the Figure and the Ground. This FE requires an image schema with a vector that extends from the Ground, or a subpart of the Ground, in some direction, and the Distance is measured along this vector from the Ground to the profiled region. In cases where the target does not introduce such an image schema, Distance should be annotated as part of the Direction FE that introduces such a vector. Note that a number of frequent adverbial targets, e.g. up.adv and down.adv, identify the Ground, with the location of an implicit viewer, so Distance is effectively measured from the viewpoint location in these cases.\nDirection (Extra-Thematic): This FE identifies the Direction from a reference location (generally, the deictic center) of a path to the Figure. This FE is filled by one of a closed class of adverbs: up, down, out (meaning 'beyond the boundary around the viewpoint'), off (meaning 'disconnect between landmark and figure'), back (meaning 'along a line from front to back of landmark, and relatively inaccessible'), and, more rarely, away, along, and dialectally, back down (meaning 'down and behind').\nTime (Peripheral): The period of time during which the locative relation between Figure and Ground obtains.\nFigures (Core-Unexpressed): The Figures are items which mutually serve to identify the location of the other items.\nDeixis (Extra-Thematic): An indication of how the locative phrase relates to the grounding of the speech situation, either locating it near (here) or far (there, yonder), or specifically separated from the speech context by distance or a barrier (over). Fillers of this FE are restricted to this closed class of items.\nAccessibility (Extra-Thematic): An indication of how accessible the Figure is to some participant in the speech context (either the speaker or the addressee, or some other highly salient viewpoint in a narrative). This FE normally describes availability for manipulation and use, but may simply describe how easily seen or difficult to see the Figure is. There is a closed class of items that can fill this slot, including only 'just' and 'right' indicating ease of access, and 'way', 'all the way', or (dialectally) 'clear' indicating distance and difficulty of access.\nDirectness (Extra-Thematic): An indication of how closely the position of Figure matches the prototype expected from the image schema associated with the target. This FE is normally filled by closed-class, dedicated adverbs like right, straight, directly, and due (and some combinations of these), but also occurs with rarer items like 'smack dab'.\nTemporal_profile (Extra-Thematic): A description of how the access path to the Ground interacts with time. This FE is filled only by the closed class of adverbials back.adv (which indicates that the position was occupied by the Figure before) and on.adv (which indicates a continued path). While this FE in its most basic use refers to cases of actual motion, it can be used with descriptions of static position, in which case it either refers to fictive, imagined motion (as in the first example below), or it backgrounds motion entirely (as in the second example).\nRegion_quantification (Extra-Thematic): A description of how much of a potential profiled region actually has (or is expected to have) a Figure in it. It is normally filled by closed-class items like 'all', 'somewhere', 'nowhere', etc.\nProfiled_region (Core): The region within which a Figure might be found. This FE is exclusive of Figure, and is a reconstrual that is not possible with verb lexical units.\n\n\n### Input:\nThe bank will not lend money **to** the company.\n<|im_end|><|im_start|>assistant<|im_sep|>", '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Goal\nFrame Definition: A Landmark (in combination with the image schema evoked by particular targets) serves to pick out the final location of a Trajector in a construed or actual motion event.\n\nFrame Elements:\nLandmark (Core): The relatively locatable entity which serves as a basis for determining the location of the Trajector using the Profiled_region.\nProfiled_region (Core-Unexpressed): The location (located with respect to the Landmark) which is profiled by the particular image schema pertaining to the locative relation.\nTrajector (Core): The entity which is construed to be in motion, whose location at the end of the construed motion event is in question.\nTime (Peripheral): The time at which the Trajector is located in the Profiled_region.\n\n\n### Input:\nThe bank will not lend money **to** the company.\n<|im_end|><|im_start|>assistant<|im_sep|>', '<|im_start|>system<|im_sep|>### Task:\nYou are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. \n\n### Notes:\n- Return the tagged sentence in a ```json ``` code block.\n- Texts must not overlap.\n<|im_end|><|im_start|>user<|im_sep|>### Frame Information:\nFrame Name: Businesses\nFrame Definition: A Proprietor owns or runs a Business which provides a Product (which may be goods or services).\n\nFrame Elements:\nProduct (Peripheral): Product identifies the product or service provided by the business.\nProprietor (Peripheral): Proprietor is the owner or chief participant in the business.\nDescriptor (Peripheral): Descriptor indicates other characteristics of the business besides its Product and Proprietor.\nPlace (Peripheral): This FE identifies the Place in which the business is located.\nBusiness (Core): Business identifies the company or establishment. This may be the target itself or the Business_Name if it is an appositive not set off with commas (i.e. restrictive):\nBusiness_name (Peripheral): The name of the Business when it appears in an appositive set off by commas (non-restrictive):\nService_provider (Peripheral): When a Business is described in terms of the professionals who work to produce the product or service, use Service_provider.\n\n\n### Input:\nThe bank will not lend money to the **company**.\n<|im_end|><|im_start|>assistant<|im_sep|>']


In [5]:
from vllm.lora.request import LoRARequest
fsp_lora = LoRARequest("fsp_lora", 1, LORA_PATH)

sampling_params = SamplingParams(
    max_tokens=MAX_TOKENS,
)

responses = llm.generate(
    messages,
    sampling_params=sampling_params,
    lora_request=fsp_lora
)

INFO 08-07 13:09:20 [__init__.py:239] Automatically detected platform cuda.


NameError: name 'LORA_PATH' is not defined